# Analyzing NYC CitiBike with RAPIDS

In this notebook, we will download and perform ETL and EDA one the NYC Citibike data using RAPIDS. 

RAPIDS is a suite of GPU accelerated data science libraries with APIs that should be familiar to users of Pandas, Scikitlearn, geopandas.  Here we'll be using the cuDF, cuML, and cuSpatial libraries, along with cuXFilter for some Visual EDA.

In [1]:
import numpy as np
import cupy
import cudf
import xgboost as xgb
import cuspatial
import os
import urllib.request

# Downloading the Data

Let's pull the raw NYC CitiBike data from their AWS S3 bucket.

In [2]:
data_dir = './bike/'
if not os.path.exists(data_dir):
    print('creating bike directory')
    os.system('mkdir -p ./bike')

Depending on your GPU size or download speed, you may have to adjust the `endyear` or even months your pull from S3.  

If you have a: 
- 16GB GPU do 1 year
- 32GB card, do 2 years
- want to adjust the notebook for `dask_cudf`, ad have all the time in the world, go until the current year

Please adjust your years accordingly

In [3]:
endyear = 2015 # 1 year

In [4]:
# download CitiBike Data
base_url = 'https://s3.amazonaws.com/tripdata/'
years = list(range(2014, endyear))
for year in years:
    for month in range(1,13):
        fn=str(year)+'{:02}'.format(month)+'-citibike-tripdata.zip'
        if not os.path.isfile(data_dir+fn):
            print(f'Downloading {base_url+fn} to {data_dir+fn}')
            urllib.request.urlretrieve(base_url+fn, data_dir+fn)

## Ingest Data 
Read one month or read all:

**Single Month**

In [5]:
#cb_df = cudf.read_csv("./bike/201401-citibike-tripdata.zip", compression="zip")

In [6]:
# cb_df = None

**Multiple Months**

In [7]:
for year in years:
    for month in range(1,13):
        fn=str(year)+'{:02}'.format(month)+'-citibike-tripdata.zip'
        try:
            tdf = cudf.read_csv(data_dir+fn, compression="zip")
            #print(tdf.head(49))
            cb_df = cudf.concat([cb_df, tdf], ignore_index=True, sort=False)
        except:
            cb_df = cudf.read_csv(data_dir+fn, compression="zip")

**All Months with `dask_cudf`**

In [8]:
# import dask
# import dask_cudf
# from dask.distributed import Client, wait
# from dask.utils import parse_bytes
# from dask_cuda import LocalCUDACluster

# cluster = LocalCUDACluster()

# client = Client(cluster)
# client
#cb_df = dask_cudf.read_csv("./bike/20*.zip", compression="zip")

Let's check out your GPU usage after reading the data.  

In [9]:
!nvidia-smi

Wed Mar 16 04:26:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.156.00   Driver Version: 450.156.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:15:00.0 Off |                  Off |
| 30%   44C    P2    44W / 250W |   6808MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GV100        Off  | 00000000:2D:00.0 Off |                  Off |
| 34%   

## ETL
Let's check out this data...

In [10]:
cb_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,471,2014-01-01 00:00:06,2014-01-01 00:07:57,2009,Catherine St & Monroe St,40.71117444,-73.99682619,263,Elizabeth St & Hester St,40.71729,-73.996375,16379,Subscriber,1986,1
1,1494,2014-01-01 00:00:38,2014-01-01 00:25:32,536,1 Ave & E 30 St,40.74144387,-73.97536082,259,South St & Whitehall St,40.70122128,-74.01234218,15611,Subscriber,1963,1
2,464,2014-01-01 00:03:59,2014-01-01 00:11:43,228,E 48 St & 3 Ave,40.7546011026,-73.971878855,2022,E 59 St & Sutton Pl,40.75849116,-73.95920622,16613,Subscriber,1991,1
3,373,2014-01-01 00:05:15,2014-01-01 00:11:28,519,Pershing Square N,40.75188406,-73.97770164,526,E 33 St & 5 Ave,40.74765947,-73.98490707,15938,Subscriber,1989,1
4,660,2014-01-01 00:05:18,2014-01-01 00:16:18,83,Atlantic Ave & Fort Greene Pl,40.68382604,-73.97632328,436,Hancock St & Bedford Ave,40.68216564,-73.95399026,19830,Subscriber,1990,1


In [11]:
cb_df.count()

tripduration               8081216
starttime                  8081216
stoptime                   8081216
start station id           8081216
start station name         8081216
start station latitude     8081216
start station longitude    8081216
end station id             8081216
end station name           8081216
end station latitude       8081216
end station longitude      8081216
bikeid                     8081216
usertype                   8081216
birth year                 7868282
gender                     8081216
dtype: int64

In [12]:
cb_df.dtypes

tripduration               object
starttime                  object
stoptime                   object
start station id           object
start station name         object
start station latitude     object
start station longitude    object
end station id             object
end station name           object
end station latitude       object
end station longitude      object
bikeid                     object
usertype                   object
birth year                 object
gender                     object
dtype: object

Everything read in as an object dtype.  Some things are, but let's start to modify some of the colums to make them more useful...

In [13]:
cb_df["end station id"] = cb_df["end station id"].astype("int16")
cb_df["start station id"] = cb_df["start station id"].astype("int16")
cb_df['gender'] = cb_df['gender'].astype("int16")
cb_df['tripduration'] = cb_df['tripduration'].astype("int16")
cb_df['bikeid'] = cb_df['bikeid'].astype("int16")

In [14]:
cb_df.dtypes

tripduration                int16
starttime                  object
stoptime                   object
start station id            int16
start station name         object
start station latitude     object
start station longitude    object
end station id              int16
end station name           object
end station latitude       object
end station longitude      object
bikeid                      int16
usertype                   object
birth year                 object
gender                      int16
dtype: object

## Data Forming

Let's pull out some interesting features from the data.  The datetime is easy pull apart using the datetime accessor

In [15]:
cb_df['starttime']=cb_df['starttime'].astype('datetime64[s]')
cb_df['stoptime']=cb_df['stoptime'].astype('datetime64[s]')
cb_df['day_of_the_month']=cb_df['starttime'].dt.day
cb_df['start_hour_of_the_day']= cb_df['starttime'].dt.hour
cb_df['stop_hour_of_the_day']= cb_df['stoptime'].dt.hour
cb_df['dow']= cb_df['starttime'].dt.dayofweek
cb_df['month']=cb_df['starttime'].dt.month
cb_df['day_of_week'] = cb_df['starttime'].dt.weekday
cb_df['is_weekend'] = (cb_df['day_of_week']>=5).astype('int32')

In [16]:
cb_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,usertype,birth year,gender,day_of_the_month,start_hour_of_the_day,stop_hour_of_the_day,dow,month,day_of_week,is_weekend
0,471,2014-01-01 00:00:06,2014-01-01 00:07:57,2009,Catherine St & Monroe St,40.71117444,-73.99682619,263,Elizabeth St & Hester St,40.71729,...,Subscriber,1986,1,1,0,0,2,1,2,0
1,1494,2014-01-01 00:00:38,2014-01-01 00:25:32,536,1 Ave & E 30 St,40.74144387,-73.97536082,259,South St & Whitehall St,40.70122128,...,Subscriber,1963,1,1,0,0,2,1,2,0
2,464,2014-01-01 00:03:59,2014-01-01 00:11:43,228,E 48 St & 3 Ave,40.7546011026,-73.971878855,2022,E 59 St & Sutton Pl,40.75849116,...,Subscriber,1991,1,1,0,0,2,1,2,0
3,373,2014-01-01 00:05:15,2014-01-01 00:11:28,519,Pershing Square N,40.75188406,-73.97770164,526,E 33 St & 5 Ave,40.74765947,...,Subscriber,1989,1,1,0,0,2,1,2,0
4,660,2014-01-01 00:05:18,2014-01-01 00:16:18,83,Atlantic Ave & Fort Greene Pl,40.68382604,-73.97632328,436,Hancock St & Bedford Ave,40.68216564,...,Subscriber,1990,1,1,0,0,2,1,2,0


In [17]:
cb_df.dtypes

tripduration                       int16
starttime                  datetime64[s]
stoptime                   datetime64[s]
start station id                   int16
start station name                object
start station latitude            object
start station longitude           object
end station id                     int16
end station name                  object
end station latitude              object
end station longitude             object
bikeid                             int16
usertype                          object
birth year                        object
gender                             int16
day_of_the_month                   int16
start_hour_of_the_day              int16
stop_hour_of_the_day               int16
dow                                int16
month                              int16
day_of_week                        int16
is_weekend                         int32
dtype: object

We can also get some data from the station location info.  Let's get the distances between the start and end stations.  To do this, we will use cuSpatial, another RAPIDS library, 

In [18]:
def haversine_dist(df):
    df['h_distance']= cuspatial.haversine_distance(
        df['start station latitude'], 
        df['start station longitude'],
        df['end station latitude'],
        df['end station longitude']
    )
    df['h_distance']= df['h_distance'].astype('float32')
    return df

cb_df = haversine_dist(cb_df)
cb_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,birth year,gender,day_of_the_month,start_hour_of_the_day,stop_hour_of_the_day,dow,month,day_of_week,is_weekend,h_distance
0,471,2014-01-01 00:00:06,2014-01-01 00:07:57,2009,Catherine St & Monroe St,40.71117444,-73.99682619,263,Elizabeth St & Hester St,40.71729,...,1986,1,1,0,0,2,1,2,0,0.194074
1,1494,2014-01-01 00:00:38,2014-01-01 00:25:32,536,1 Ave & E 30 St,40.74144387,-73.97536082,259,South St & Whitehall St,40.70122128,...,1963,1,1,0,0,2,1,2,0,4.293091
2,464,2014-01-01 00:03:59,2014-01-01 00:11:43,228,E 48 St & 3 Ave,40.7546011026,-73.971878855,2022,E 59 St & Sutton Pl,40.75849116,...,1991,1,1,0,0,2,1,2,0,1.414189
3,373,2014-01-01 00:05:15,2014-01-01 00:11:28,519,Pershing Square N,40.75188406,-73.97770164,526,E 33 St & 5 Ave,40.74765947,...,1989,1,1,0,0,2,1,2,0,0.811626
4,660,2014-01-01 00:05:18,2014-01-01 00:16:18,83,Atlantic Ave & Fort Greene Pl,40.68382604,-73.97632328,436,Hancock St & Bedford Ave,40.68216564,...,1990,1,1,0,0,2,1,2,0,2.483842


### EDA Dataviz 

We'll start with using cuxfilter.  What we want to do is put this data into context.  As this is Longitde and latitude, we'll be using a scatter plot overlayed on a map.  

In [19]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import double_feature
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import Hot
from pyproj import Proj, Transformer

In [20]:
def latlong2tile(x, y):
    temp= cudf.DataFrame()
    transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
    temp['x'], temp['y'] = transform_4326_to_3857.transform(
                                                x.to_array(), y.to_array()
                                            )
    print(temp.head())
    return temp.x, temp.y

In [21]:
cb_df['sslat'], cb_df['sslong'] = latlong2tile(cb_df['start station latitude'], cb_df['start station longitude'])
cb_df['eslat'], cb_df['eslong'] = latlong2tile(cb_df['end station latitude'],cb_df['end station longitude'])

/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/series.py:1709: FutureWarning: The to_array method will be removed in a future cuDF release. Consider using `to_numpy` instead.
  warnings.warn(


              x             y
0 -8.237289e+06  4.969833e+06
1 -8.234899e+06  4.974279e+06
2 -8.234512e+06  4.976212e+06
3 -8.235160e+06  4.975813e+06
4 -8.235007e+06  4.965817e+06


/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/series.py:1709: FutureWarning: The to_array method will be removed in a future cuDF release. Consider using `to_numpy` instead.
  warnings.warn(


              x             y
0 -8.237239e+06  4.970731e+06
1 -8.239016e+06  4.968371e+06
2 -8.233101e+06  4.976784e+06
3 -8.235962e+06  4.975192e+06
4 -8.232521e+06  4.965574e+06


With our new columns of mapped points, let's create our charts.  This is going to be a bit different than your standard visualization...this viz is iteractive!  You'll actually get to play with and explore your data!

In [22]:
cux = cuxfilter.DataFrame.from_dataframe(cb_df)
chart0 = cuxfilter.charts.scatter(x='sslat',
                                      y='sslong',
                                      title='NYC Bike Pickups',
                                      aggregate_fn='count',
                                      tile_provider="CartoLight", x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55),
                                  pixel_shade_type="linear",
                                  color_palette=palettes.viridis(10)
                                 )
chart1 = cuxfilter.charts.scatter(x='eslat',
                                      y='eslong',
                                      title='NYC Bike Dropoffs',
                                      aggregate_fn='count',
                                      tile_provider="CartoLight", x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55),
                                 pixel_shade_type="linear",
                                 color_palette=palettes.viridis(10))
chart2 = cuxfilter.charts.multi_select('month')
chart3 = cuxfilter.charts.multi_select('dow')
chart4 = cuxfilter.charts.multi_select('start_hour_of_the_day')
# chart5 = cuxfilter.charts.multi_select('start station id')
# chart6 = cuxfilter.charts.multi_select('end station id')
chart7 = cuxfilter.charts.range_slider('h_distance')

In [23]:
d = cux.dashboard([chart0, chart1], sidebar=[chart2, 
                                             chart3, 
                                             chart4, 
                                             # chart5, 
                                             # chart6, 
                                             chart7], 
                  layout=cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.dark, title= 'NYC CITIBIKE DATASET')

### For Fun:
What interesting insights can you discover?

Try checking out what `h_distance_range_slider` to 5km to max distance (7.13 for 2014).  What's interesting is that you can see that the pick up and drop off distances are in the same areas.  Playing around with the hours, you can see that just over 1/3 occur in pre-noon hours, and under 2/3s occur in the afternoon and night. Most rides seem to be commuter rides during the work week, happening during the warmer months.  These rides seem to occur in 4 main clusters.

If you want to see which areas go where, you can create two more range sliders for `start station latitude` and `start station longitude` so that you create boxes around certain latitudes and longitudes

In [24]:
d.app()

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=280)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=8081216)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='month_multi_select', width=400)
            [0] MultiSelect(height=200, options=[1, 2, 3, 4, 5, ...], sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='dow_multi_select', width=400)
            [0] MultiSelect(height=200, options=[0, 1, 2, 3, 4, 5, 6, ''], sizing_mode='stretch_both', value=[''], width=400)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='start_hour_of_the_day_mul..., width=400)
            [0] MultiSelect(height=200, options=[0, 1, 2, 3, 4, ...], sizing_mode='stretch_both', value=[''], width=400)
        [4] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='h_distance_range_slider', width=400)
            [0] RangeSlider(end=7.133639, sizing_mode='stretch_both', start=3.3535125e-16, value=(3.3535125e-16, 7.133639), value_end=7.133639, value_start=3.3535125e-16)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='NYC Bike Pickups', width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')
        [1] Card(sizing_mode='stretch_both', title='NYC Bike Dropoffs', width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

cuXFilter has the ability to export what you see.  While we won't do anything interesting with this data, you could perform the next stage 

In [26]:
cux_df = d.export()

final query @h_distance_min<=h_distance<=@h_distance_max


In [27]:
cux_df.head(49)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,stop_hour_of_the_day,dow,month,day_of_week,is_weekend,h_distance,sslat,sslong,eslat,eslong
458,2363,2014-01-01 02:53:14,2014-01-01 03:32:37,373,Willoughby Ave & Walworth St,40.69331716,-73.95381995,482,W 15 St & 7 Ave,40.73935542,...,3,2,1,2,0,5.252768,-8.232502e+06,4.967211e+06,-8.237566e+06,4.973972e+06
2815,2286,2014-01-01 14:42:44,2014-01-01 15:20:50,358,Christopher St & Greenwich St,40.73291553,-74.00711384,539,Metropolitan Ave & Bedford Ave,40.71534825,...,15,2,1,2,0,5.239797,-8.238434e+06,4.973026e+06,-8.233216e+06,4.970446e+06
5690,2273,2014-01-01 21:30:15,2014-01-01 22:08:08,290,2 Ave & E 58 St,40.76020258,-73.96478473,3002,South End Ave & Liberty St,40.711512,...,22,2,1,2,0,5.861149,-8.233722e+06,4.977036e+06,-8.239396e+06,4.969882e+06
5828,1490,2014-01-01 22:13:19,2014-01-01 22:38:09,532,S 5 Pl & S 4 St,40.710451,-73.960876,346,Bank St & Hudson St,40.73652889,...,22,2,1,2,0,5.100742,-8.233287e+06,4.969727e+06,-8.238330e+06,4.973557e+06
6676,1359,2014-01-02 07:28:49,2014-01-02 07:51:28,354,Emerson Pl & Myrtle Ave,40.69363137,-73.96223558,417,Barclay St & Church St,40.71291224,...,7,3,1,3,0,5.366353,-8.233438e+06,4.967257e+06,-8.238778e+06,4.970088e+06
6790,1210,2014-01-02 07:40:40,2014-01-02 08:00:50,532,S 5 Pl & S 4 St,40.710451,-73.960876,337,Old Slip & Front St,40.7037992,...,8,3,1,3,0,5.286895,-8.233287e+06,4.969727e+06,-8.238576e+06,4.968750e+06
8063,1552,2014-01-02 09:11:30,2014-01-02 09:37:22,532,S 5 Pl & S 4 St,40.710451,-73.960876,337,Old Slip & Front St,40.7037992,...,9,3,1,3,0,5.286895,-8.233287e+06,4.969727e+06,-8.238576e+06,4.968750e+06
8159,1284,2014-01-02 09:17:59,2014-01-02 09:39:23,443,Bedford Ave & S 9th St,40.70853074,-73.96408963,319,Park Pl & Church St,40.71336124,...,9,3,1,3,0,5.037818,-8.233645e+06,4.969445e+06,-8.238686e+06,4.970154e+06
8802,1653,2014-01-02 10:25:19,2014-01-02 10:52:52,532,S 5 Pl & S 4 St,40.710451,-73.960876,259,South St & Whitehall St,40.70122128,...,10,3,1,3,0,5.729777,-8.233287e+06,4.969727e+06,-8.239016e+06,4.968371e+06
11289,1221,2014-01-02 15:43:27,2014-01-02 16:03:48,351,Front St & Maiden Ln,40.70530954,-74.00612572,532,S 5 Pl & S 4 St,40.710451,...,16,3,1,3,0,5.034011,-8.238324e+06,4.968972e+06,-8.233287e+06,4.969727e+06


In [28]:
cux_df["birth year"].value_counts()

\N      1202
1977    1188
1984    1033
1981    1020
1986     805
        ... 
1942       3
1939       2
1943       2
1938       1
1940       1
Name: birth year, dtype: int32

While most of these riders are in thier 40s, or are non-Subscribing Customers (they don't have to share their age).  Tourists ;).  We'll have to do something about those `\N's` soon.

However, those bottom few are interesting.  1938?  Seriously?  76 years old at the time...  

In [29]:
cb_df['birth year'] = cb_df['birth year'].replace("\\N", None)
cb_df = cb_df.dropna()

In [30]:
cb_df.dtypes

tripduration                       int16
starttime                  datetime64[s]
stoptime                   datetime64[s]
start station id                   int16
start station name                object
start station latitude            object
start station longitude           object
end station id                     int16
end station name                  object
end station latitude              object
end station longitude             object
bikeid                             int16
usertype                          object
birth year                        object
gender                             int16
day_of_the_month                   int16
start_hour_of_the_day              int16
stop_hour_of_the_day               int16
dow                                int16
month                              int16
day_of_week                        int16
is_weekend                         int32
h_distance                       float32
sslat                            float64
sslong          

In [31]:
cb_df['birth year']=cb_df['birth year'].astype("int16")
cb_df['gender'] = cb_df['gender'].astype("int16")

## Make Your Own

With that cleared up, use the space below to use cuXFilter to make an age based histogram.  If you need help, please check out the docs here: https://docs.rapids.ai/api/cuxfilter/stable/charts/bokeh_charts.html

# Train the XGBoost Regression Model

I wonder how well we can predict the age of the rider by the 
- Length of the ride
- distance travelled
- day and time of the ride
- location

Now let's set up ths XGBoost Regression training.

In [32]:
from cuml.preprocessing import train_test_split

columns = ["dow",
           "month",
           "day_of_week",
           "is_weekend",
           "tripduration",
           "start_hour_of_the_day",
            "stop_hour_of_the_day",
            "start station id",
            "end station id",
            "birth year",
            "h_distance",
            "day_of_the_month",
            "gender",
            "bikeid"
          ]

cb_df = cb_df[columns]

X_train, X_test, y_train, y_test = train_test_split(cb_df, "birth year", train_size=0.8)

In [33]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

The wall time output below indicates how long it took your GPU cluster to train an XGBoost model over the training set.

In [34]:
%%time

params = {
    'learning_rate': 0.3,
    'max_depth': 9,
    'objective': 'reg:squarederror',
    'subsample': 0.8,
    'gamma': 1,
    'silent': False,
    'verbose_eval': True,
    'tree_method':'gpu_hist'
}

trained_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train')]
)

[04:28:55] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1643825216234/work/src/learner.cc:576: 
Parameters: { "silent", "verbose_eval" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:1382.92737
[1]	train-rmse:968.08264
[2]	train-rmse:677.70544
[3]	train-rmse:474.46213
[4]	train-rmse:332.22055
[5]	train-rmse:232.69420
[6]	train-rmse:163.08299
[7]	train-rmse:114.43773
[8]	train-rmse:80.49995
[9]	train-rmse:56.90940
[10]	train-rmse:40.62050
[11]	train-rmse:29.51797
[12]	train-rmse:22.12557
[13]	train-rmse:17.38387
[14]	train-rmse:14.49160
[15]	train-rmse:12.84480
[16]	train-rmse:11.93867
[17]	train-rmse:11.46285
[18]	train-rmse:11.21549
[19]	train-rmse:11.09160
[20]	train-rmse:11.03027
[21]	train-rmse:10.97842
[22]	train-rmse:10.95

In [35]:
prediction = trained_model.predict(dtest)

In [36]:
df = X_test.copy()

df['actual'] = y_test.values
df['predicted'] = prediction.astype("int16")

df.tail()

,dow,month,day_of_week,is_weekend,tripduration,start_hour_of_the_day,stop_hour_of_the_day,start station id,end station id,h_distance,day_of_the_month,gender,bikeid,actual,predicted
4695502,5,8,5,1,2012,20,21,238,367,4.268119,9,2,19457,1974,1974
7113955,5,6,5,1,729,4,4,260,428,2.728507,23,1,14863,1969,1976
3119535,5,6,5,1,186,10,10,494,489,0.514136,21,2,17856,1985,1983
2321993,0,5,0,0,1392,12,12,305,457,1.617125,26,1,18550,1980,1977
1350895,4,4,4,0,125,13,13,360,316,0.269898,18,1,18413,1980,1977


Form and test predictions from xgboost output

In [37]:
## MSE requires values be float32 
y_test = y_test.astype(np.float32) 

## Test prediction wih RMSE, compare it to sklearn and pandas.  
from cuml.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test.values, prediction))

## RMSE for the age prediction.  Let's see what we get...
print("RMSE: ", rmse)

RMSE:  10.418495


10 years is about a standard age range deviation

# Take the model with you

In [38]:
import joblib

# Save the booster to file
joblib.dump(trained_model, "xgboost-model")

['xgboost-model']

## Reload a Saved Model from Disk

You can also read the saved model back into a normal XGBoost model object.

In [39]:
with open("xgboost-model", 'rb') as fh:  
    loaded_model = joblib.load(fh)

In [40]:
# Generate predictions on the test set again, but this time using the reloaded model
new_preds = loaded_model.predict(dtest)

# Verify that the predictions result in the same RMSE error
rmse = np.sqrt(mean_squared_error(y_test.values, new_preds))

## RMSE for the age prediction.  Let's see what we get...
print("RMSE: ", rmse)

RMSE:  10.418495
